In [1]:
import pandas as pd
from functions import custom_cv_eval_v2, get_ev_from_df
import random
import json
import numpy as np


In [2]:
%run MODEL_LIST.ipynb            #Load the models

In [3]:
df=pd.read_csv("data/owl-with-odds.csv")

In [4]:
#The odds need to be changed to numeric...

subset = ['t1_odds', 't2_odds']
df.dropna(subset=subset ,inplace=True)
df['t1_odds'] = pd.to_numeric(df['t1_odds'], errors='coerce')
df['t2_odds'] = pd.to_numeric(df['t2_odds'], errors='coerce')
df.dropna(subset=subset ,inplace=True)
#Create the odds df
odds_df = df[subset]

#Create a label df
label_df = df['winner_label']

In [5]:
#Split the test set.  We are always(?) going to use the last 60 matches as the test set, so we don't want those around
#as we pick models
df_train = df[:-60]
odds_train = odds_df[:-60]
label_train = label_df[:-60]

df_test = df[-60:]
odds_test = odds_df[-60:]
label_test = label_df[-60:]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

60
60
60
527
527
527


In [6]:
display(df_train)
display(df_test)

,Unnamed: 0,Unnamed: 0.1,id,team_one,team_two,stage,winner,date,corona_virus_isolation,t1_wins_season,...,t2_win_percent_last_20,t1_place_last_season,t2_place_last_season,t1_wins_vs_t2,t1_losses_vs_t2,t1_matches_vs_t2,t1_win_percent_vs_t2,winner_label,t1_odds,t2_odds
1,1,1,10224,Los Angeles Gladiators,Shanghai Dragons,Overwatch League - Stage 1,Los Angeles Gladiators,1/11/2018,False,0.0,...,NaN,NaN,NaN,0,0,0,NaN,0,-370.0,276.0
2,2,2,10225,Seoul Dynasty,Dallas Fuel,Overwatch League - Stage 1,Seoul Dynasty,1/11/2018,False,0.0,...,NaN,NaN,NaN,0,0,0,NaN,0,-588.0,413.0
3,3,3,10226,Florida Mayhem,London Spitfire,Overwatch League - Stage 1,London Spitfire,1/11/2018,False,0.0,...,NaN,NaN,NaN,0,0,0,NaN,1,652.0,-1667.0
4,4,4,10227,Houston Outlaws,Philadelphia Fusion,Overwatch League - Stage 1,Philadelphia Fusion,1/11/2018,False,0.0,...,NaN,NaN,NaN,0,0,0,NaN,1,-161.0,143.0
5,5,5,10228,New York Excelsior,Boston Uprising,Overwatch League - Stage 1,New York Excelsior,1/12/2018,False,0.0,...,NaN,NaN,NaN,0,0,0,NaN,0,-769.0,380.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,637,637,33081,Chengdu Hunters,Vancouver Titans,OWL 2020 Regular Season,Chengdu Hunters,4/12/2020,True,1.0,...,0.70,12.0,1.0,0,2,2,0.000000,0,167.0,-222.0
638,638,638,32852,Shanghai Dragons,Guangzhou Charge,OWL 2020 Regular Season,Shanghai Dragons,4/12/2020,True,3.0,...,0.65,11.0,9.0,2,1,3,0.666667,0,-323.0,235.0
639,639,639,31093,Houston Outlaws,Toronto Defiant,OWL 2020 Regular Season,Houston Outlaws,4/12/2020,True,3.0,...,0.20,16.0,17.0,2,1,3,0.666667,0,131.0,-172.0
640,640,640,31064,Philadelphia Fusion,Atlanta Reign,OWL 2020 Regular Season,Philadelphia Fusion,4/12/2020,True,7.0,...,0.65,10.0,6.0,3,0,3,1.000000,0,-192.0,147.0


,Unnamed: 0,Unnamed: 0.1,id,team_one,team_two,stage,winner,date,corona_virus_isolation,t1_wins_season,...,t2_win_percent_last_20,t1_place_last_season,t2_place_last_season,t1_wins_vs_t2,t1_losses_vs_t2,t1_matches_vs_t2,t1_win_percent_vs_t2,winner_label,t1_odds,t2_odds
642,642,642,32862,Atlanta Reign,Washington Justice,OWL 2020 Regular Season,Atlanta Reign,4/16/2020,True,3.0,...,0.40,6.0,17.0,2,0,2,1.000000,0,-1111.0,606.0
643,643,643,32861,Los Angeles Gladiators,Los Angeles Valiant,OWL 2020 Regular Season,Los Angeles Gladiators,4/17/2020,True,2.0,...,0.55,5.0,13.0,3,4,7,0.428571,0,-222.0,166.0
644,644,644,32856,Florida Mayhem,Paris Eternal,OWL 2020 Regular Season,Florida Mayhem,4/17/2020,True,2.0,...,0.45,20.0,14.0,0,2,2,0.000000,0,250.0,-345.0
645,645,645,32855,Boston Uprising,Toronto Defiant,OWL 2020 Regular Season,Toronto Defiant,4/18/2020,True,1.0,...,0.20,19.0,17.0,1,2,3,0.333333,1,200.0,-278.0
646,646,646,32853,Dallas Fuel,San Francisco Shock,OWL 2020 Regular Season,San Francisco Shock,4/18/2020,True,2.0,...,0.80,15.0,3.0,1,6,7,0.142857,1,586.0,-1111.0
647,647,647,32851,Chengdu Hunters,Guangzhou Charge,OWL 2020 Regular Season,Chengdu Hunters,4/18/2020,True,2.0,...,0.65,12.0,9.0,1,3,4,0.250000,0,147.0,-196.0
648,648,648,32845,Shanghai Dragons,Hangzhou Spark,OWL 2020 Regular Season,Shanghai Dragons,4/18/2020,True,4.0,...,0.60,11.0,4.0,1,2,3,0.333333,0,-455.0,312.0
649,649,649,32844,Guangzhou Charge,Hangzhou Spark,OWL 2020 Regular Season,Guangzhou Charge,4/19/2020,True,3.0,...,0.55,9.0,4.0,2,2,4,0.500000,0,100.0,-132.0
650,650,650,32843,Chengdu Hunters,Shanghai Dragons,OWL 2020 Regular Season,Shanghai Dragons,4/19/2020,True,3.0,...,0.55,12.0,11.0,1,3,4,0.250000,1,277.0,-400.0
651,651,651,32842,Seoul Dynasty,Hangzhou Spark,OWL 2020 Regular Season,Seoul Dynasty,4/25/2020,True,2.0,...,0.50,8.0,4.0,0,3,3,0.000000,0,-405.0,360.0


In [7]:
def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels,
                  test_odds, verbose=True):
    model_score = 0
    
    df_train = train_df[input_features].copy()
    df_test = test_df[input_features].copy()
    df_train = df_train.dropna()
    df_test = df_test.dropna()
        
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)
    df_train, df_test = df_train.align(df_test, join='left', axis=1)    #Ensures both sets are dummified the same
    df_test = df_test.fillna(0)

    #LOOK AT get_ev and prepare the labels and odds
    
    labels_train = train_labels[train_labels.index.isin(df_train.index)]
    odds_train = train_odds[train_odds.index.isin(df_train.index)] 
    labels_test = test_labels[test_labels.index.isin(df_test.index)]
    odds_test = test_odds[test_odds.index.isin(df_test.index)] 
    
    
    
    display(df_train.shape)
    display(labels_train.shape)
    display(odds_train.shape)
    display(df_test.shape)
    display(labels_test.shape)
    display(odds_test.shape)
    
    input_model.fit(df_train, labels_train)

    
    
    probs = input_model.predict_proba(df_test)

    
    odds_test = np.array(odds_test)    
    
    
    prepped_test = list(zip(odds_test[:, -2], odds_test[:, -1], probs[:, 0], probs[:, 1], labels_test))
    ev_prepped_df = pd.DataFrame(prepped_test, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
    
    display(ev_prepped_df)
    
    #display(df_test)
    #display(df_test)
    model_score = get_ev_from_df(ev_prepped_df, print_stats = True, min_ev = input_ev, get_total=True)
    
    return(model_score)

In [8]:
m1_score = evaluate_model(model_1, features_1, min_ev_1, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

(449, 6)

(449,)

(449, 2)

(60, 6)

(60,)

(60, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-1111.0,606.0,0.815762,0.184238,0
1,-222.0,166.0,0.716029,0.283971,0
2,250.0,-345.0,0.425873,0.574127,0
3,200.0,-278.0,0.705765,0.294235,1
4,586.0,-1111.0,0.390178,0.609822,1
5,147.0,-196.0,0.689883,0.310117,0
6,-455.0,312.0,0.437228,0.562772,0
7,100.0,-132.0,0.741072,0.258928,0
8,277.0,-400.0,0.685609,0.314391,1
9,-405.0,360.0,0.635723,0.364277,0



          Number of matches: 60
          Number of bets: 58
          Number of winning bets: 17
          Number of losing bets: 41
          Number of underdog bets: 54
          Number of underdog wins: 13
          Number of underdog losses: 41
          Number of Favorite bets: 4
          Number of favorite wins: 4
          Number of favorite losses: 0
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: 115.03036548901218
          Profit per bet: 1.9832821636036584
          Profit per match: 1.9171727581502032
          
          
# Matches: 60, # Bets: 58 # Wins: 17


In [9]:
m2_score = evaluate_model(model_2, features_2, min_ev_2, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

(455, 4)

(455,)

(455, 2)

(60, 4)

(60,)

(60, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-1111.0,606.0,0.000000,1.000000,0
1,-222.0,166.0,0.000000,1.000000,0
2,250.0,-345.0,0.416667,0.583333,0
3,200.0,-278.0,0.214286,0.785714,1
4,586.0,-1111.0,0.090909,0.909091,1
5,147.0,-196.0,1.000000,0.000000,0
6,-455.0,312.0,0.000000,1.000000,0
7,100.0,-132.0,0.500000,0.500000,0
8,277.0,-400.0,0.772727,0.227273,1
9,-405.0,360.0,0.000000,1.000000,0



          Number of matches: 60
          Number of bets: 56
          Number of winning bets: 25
          Number of losing bets: 31
          Number of underdog bets: 34
          Number of underdog wins: 8
          Number of underdog losses: 26
          Number of Favorite bets: 22
          Number of favorite wins: 17
          Number of favorite losses: 5
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: -493.24896192392004
          Profit per bet: -8.808017177212857
          Profit per match: -8.220816032065335
          
          
# Matches: 60, # Bets: 56 # Wins: 25


In [10]:
m3_score = evaluate_model(model_3, features_3, min_ev_3, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

(441, 3)

(441,)

(441, 2)

(60, 3)

(60,)

(60, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-1111.0,606.0,0.551667,0.448333,0
1,-222.0,166.0,0.691365,0.308635,0
2,250.0,-345.0,0.202483,0.797517,0
3,200.0,-278.0,0.464548,0.535452,1
4,586.0,-1111.0,0.235714,0.764286,1
5,147.0,-196.0,0.216227,0.783773,0
6,-455.0,312.0,0.512032,0.487968,0
7,100.0,-132.0,0.386313,0.613687,0
8,277.0,-400.0,0.214087,0.785913,1
9,-405.0,360.0,0.584065,0.415935,0



          Number of matches: 60
          Number of bets: 47
          Number of winning bets: 20
          Number of losing bets: 27
          Number of underdog bets: 31
          Number of underdog wins: 8
          Number of underdog losses: 23
          Number of Favorite bets: 16
          Number of favorite wins: 12
          Number of favorite losses: 4
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: 589.7114810265058
          Profit per bet: 12.547052787797996
          Profit per match: 9.828524683775097
          
          
# Matches: 60, # Bets: 47 # Wins: 20


In [11]:
m4_score = evaluate_model(model_4, features_4, min_ev_4, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

(441, 5)

(441,)

(441, 2)

(60, 5)

(60,)

(60, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-1111.0,606.0,0.731974,0.268026,0
1,-222.0,166.0,0.565571,0.434429,0
2,250.0,-345.0,0.609146,0.390854,0
3,200.0,-278.0,0.392816,0.607184,1
4,586.0,-1111.0,0.038112,0.961888,1
5,147.0,-196.0,0.592357,0.407643,0
6,-455.0,312.0,0.686376,0.313624,0
7,100.0,-132.0,0.660047,0.339953,0
8,277.0,-400.0,0.556143,0.443857,1
9,-405.0,360.0,0.686881,0.313119,0



          Number of matches: 60
          Number of bets: 54
          Number of winning bets: 23
          Number of losing bets: 31
          Number of underdog bets: 38
          Number of underdog wins: 10
          Number of underdog losses: 28
          Number of Favorite bets: 16
          Number of favorite wins: 13
          Number of favorite losses: 3
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: -44.770314117657335
          Profit per bet: -0.8290798910677284
          Profit per match: -0.7461719019609556
          
          
# Matches: 60, # Bets: 54 # Wins: 23
